In [2]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('Departamentos_y_municipios_de_Colombia_20250114.csv', encoding='utf-8')

# Exportar a un archivo Excel
df.to_excel('archivo.xlsx', index=False)